In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("retouren.csv")

**1. Data Cleaning**

In [4]:
df.describe().round(1)

,order_item_id,item_id,brand_id,item_price,user_id,return
count,75007.0,75007.0,75007.0,75007.0,75007.0,75007.0
mean,37504.0,784.8,28.8,60.7,25740.2,0.5
std,21652.8,680.0,26.8,44.7,12947.2,0.5
min,1.0,1.0,1.0,0.0,9.0,0.0
25%,18752.5,176.0,5.0,29.9,14674.0,0.0
50%,37504.0,479.0,24.0,49.9,30486.0,0.0
75%,56255.5,1517.0,43.0,74.9,36479.0,1.0
max,75007.0,2224.0,137.0,999.0,43801.0,1.0


Spalten mit Item_id von 0 

In [5]:
df[df.item_price == 0].item_id.unique()

array([ 157,   62,  312,  553,  807, 1014,  239, 1436, 1725, 1904, 1951,
       1472, 2025, 2103, 2121, 2199, 2213])

In [17]:
df[df.item_id == 157][["item_color", "item_price"]]

,item_color,item_price
202,purple,0.0
216,purple,0.0
218,purple,0.0
329,purple,9.9
351,purple,0.0
...,...,...
65632,jade,9.9
65943,jade,9.9
66731,jade,9.9
68026,jade,9.9


In [14]:
df[df.item_id == 157][["item_color", "item_price"]].groupby("item_color").agg(
 mean_price = ("item_price", "mean"),
 max_price= ("item_price", "max"),
 min_price = ("item_price", "min")
)

,mean_price,max_price,min_price
item_color,,,
jade,10.710811,19.9,9.9
purple,1.218436,19.9,0.0


In [20]:
df[(df.item_id == 157) & (df.item_color == "purple")][["item_color", "item_price"]].item_price.unique()

array([ 0. ,  9.9, 19.9])

In [26]:
df[df.item_id == 157][["item_color", "item_price"]].groupby(["item_color", "item_price"]).agg(
 item_price_count = ("item_price", "count")
)

item_price_count
item_color item_price                  
jade       9.9                       34
           19.9                       3
purple     0.0                      160
           9.9                       16
           19.9                       3